## Cleanup Previo
En el respositorio del proyecto se encuentran dos pequeños scripts de Python, `download.py` y `rewrite.py`. Estos scripts fueron ejecutados para descargar el set de datos de la página provista, y juntar los resultados (varios archivos `.csv`) en un sólo archivo, a través de `pandas`. A partir de este último, se hace un filtrado de los resultados por ubicación de las propiedades, para quedarnos solo con las del Gran Buenos Aires. Este último DataFrame es el punto de partida de nuestro análisis exploratorio, y para no tener que correr la limpieza cada vez que se trabaja sobre él, se guarda en un csv nuevo.

In [1]:
import pandas as pd
import numpy as np
import mplleaflet
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
%matplotlib inline  

## Carga de datos
Cargo solo las columnas que nos pueden llegar a interesar. En particular, ignoro varias que tienen URL a imágenes y descripciones de las propiedades individuales. 

In [3]:
dtypes = {
    'country_name': str,
    'expenses': object,
    'id': str,
    'title': str,
    'state_name': str,
    'lat': float,
    'lon': float
}
cols = range(6) + range(8, 18) + [20, 21] + range(23, 27)
df = pd.read_csv('data/data_gba_total.csv', dtype=dtypes, usecols=cols)


In [4]:
df

,country_name,created_on,currency,expenses,floor,geonames_id,lat,lat-lon,lon,operation,...,price,price_aprox_local_currency,price_aprox_usd,price_per_m2,property_type,rooms,surface_covered_in_m2,surface_in_m2,surface_total_in_m2,title
0,Argentina,2015-07-01,USD,1500.0,NaN,3430234.0,-34.574594,"-34.5745944268,-58.4192548517",-58.419255,sell,...,71000.0,1133763.50,71000.00,2448.275862,apartment,1.0,29.0,NaN,29.0,Departamento en Palermo
1,Argentina,2015-07-01,USD,1.0,NaN,3427455.0,-34.610611,"-34.6106110157,-58.4795902886",-58.479590,sell,...,96000.0,1532976.00,96000.00,2285.714286,apartment,2.0,42.0,NaN,47.0,SAN BLAS 2700 6º- A ESTRENAR
2,Argentina,2015-07-01,USD,1.0,NaN,3427455.0,-34.610611,"-34.6106110157,-58.4795902886",-58.479590,sell,...,86000.0,1373291.00,86000.00,2047.619048,apartment,2.0,42.0,NaN,47.0,SAN BLAS 2700 4º- A ESTRENAR
3,Argentina,2015-07-01,USD,NaN,NaN,3436134.0,-34.639086,"-34.639086,-58.369347",-58.369347,sell,...,165000.0,2634802.50,165000.00,2115.384615,apartment,1.0,78.0,NaN,86.0,DEPARTAMENTO EN VENTA
4,Argentina,2015-07-01,USD,NaN,NaN,3436134.0,-34.678121,"-34.6781214564,-58.3922113975",-58.392211,sell,...,85000.0,1357322.50,85000.00,1133.333333,house,NaN,75.0,NaN,65.0,Duplex en PH - Avellaneda - Excelente
5,Argentina,2015-07-02,USD,NaN,NaN,3435259.0,-34.611630,"-34.611630054,-58.4013462067",-58.401346,sell,...,56900.0,908607.65,56900.00,1625.714286,apartment,NaN,35.0,NaN,35.0,"Departamento en Venta en Congreso, Capital Fed..."
6,Argentina,2015-07-02,NaN,NaN,NaN,3430114.0,-34.644395,"-34.6443953,-58.4153267",-58.415327,sell,...,NaN,NaN,NaN,NaN,apartment,1.0,30.0,NaN,47.0,PARQUE PATRICIOS MONOAMB.EN POZO C/PATIO GUARA...
7,Argentina,2015-07-02,NaN,NaN,NaN,3430114.0,-34.644395,"-34.6443953,-58.4153267",-58.415327,sell,...,NaN,NaN,NaN,NaN,apartment,1.0,32.0,NaN,48.0,PARQUE PATRICIOS MONOAMB.EN POZO C/PATIO GUARA...
8,Argentina,2015-07-02,NaN,NaN,NaN,3430114.0,-34.644395,"-34.6443953,-58.4153267",-58.415327,sell,...,NaN,NaN,NaN,NaN,apartment,1.0,30.0,NaN,33.0,PARQUE PATRICIOS MONOAMB.EN POZO C/BALCON GUAR...
9,Argentina,2015-07-02,NaN,NaN,NaN,3430114.0,-34.644395,"-34.6443953,-58.4153267",-58.415327,sell,...,NaN,NaN,NaN,NaN,apartment,1.0,27.0,NaN,28.0,PARQUE PATRICIOS MONOAMB.EN POZO C/PATIO GUARA...


In [5]:
df_deptos_caros_capital = df.dropna(subset=['lat', 'lon'], how='any').loc[(df.property_type.str.contains('apartment'))&(df.place_name == 'Capital Federal')].sort_values('price_per_m2',ascending=False).head(100)

In [7]:
geometry = [Point(x,y) for x,y in zip(df_deptos_caros_capital['lon'], df_deptos_caros_capital['lat'])]

gs = GeoDataFrame(df_deptos_caros_capital, geometry=geometry)

In [8]:
gs.plot(marker='o', color='red', markersize=60, figsize=(4, 4))
mplleaflet.display()